In [1]:
from soda.contracts.data_contract_translator import DataContractTranslator
from soda.scan import Scan
import logging
from dotenv import load_dotenv
import os
import yaml
import pandas as pd
import json

/home/ubuntu/.local/share/virtualenvs/poc_soda-1ZRiZgOA/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
load_dotenv()
app_main_path=os.environ["APP_MAIN_PATH"]
datacontracts_path=os.environ["DATA_CONTRACTS_PATH"]
postgresql_conn_path=os.environ["POSTGRESQL_CONNECTION_YML_PATH"]

In [3]:
# Cargar contrato de datos en string Importante cambiar la ruta de dc_path hacia el contrato de datos correcto
dc_path=f"{app_main_path}/{datacontracts_path}/03_vw_dim_lob.yml"
print(f"dc_path: {dc_path}")
with open(dc_path) as infile:
    data_contract_yaml_str: str = infile.read()

# Traduce e interpreta el string del contrato de datos en el formato estandar de SodaCL
data_contract_parser = DataContractTranslator()
sodacl_yaml_str = data_contract_parser.translate_data_contract_yaml_str(data_contract_yaml_str)

# Ejecuta el contrato de datos en la conexión definida previemante en el postgresql_conn_path
scan = Scan()
scan.set_data_source_name("adp") # Importante cambiar el datasource_name a uno que exista en el postgresql_conn_path
scan.add_configuration_yaml_file(file_path=postgresql_conn_path)
scan.add_sodacl_yaml_str(sodacl_yaml_str)

# Ejecuta el escaneo definido en el contrato de datos
scan.execute()

# Guarda los resultados completos del escaneo en un diccionario
scan_results = scan.build_scan_results()

# Guarda una lista con los fallos resultadfo del escaneo
fail_list = scan.get_checks_fail()

# Guarda en formato de texto los resultados de la ejecución del contrato de datos
scan_results_string = scan.get_all_checks_text()


dc_path: /home/ubuntu/proyectos/poc_soda2/poc_soda/contracts/03_vw_dim_lob.yml


[08:07:04] Could not connect to data source "adp": connection to server at "intl-andesmcm-digital-eco-dataplatform-datamart-prod.czjshh2jedqq.us-east-1.redshift.amazonaws.com" (10.142.163.12), port 9519 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

  | connection to server at "intl-andesmcm-digital-eco-dataplatform-datamart-prod.czjshh2jedqq.us-east-1.redshift.amazonaws.com" (10.142.163.12), port 9519 failed: Connection refused
  | 	Is the server running on that host and accepting TCP/IP connections?

[08:07:04] No valid checks found, 0 checks evaluated.
[08:07:04] Could not connect to data source "adp": connection to server at "intl-andesmcm-digital-eco-dataplatform-datamart-prod.czjshh2jedqq.us-east-1.redshift.amazonaws.com" (10.142.163.12), port 9519 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

  | connection to server at "intl-andesmcm-digital-eco-dataplatform-datamart-prod.czjs

In [6]:
#!pip list | grep -i numpy
#!pip install --upgrade numpy==1.26.4
#!pip install requests
#!pip install psycopg2-binary
!pip install --upgrade soda-core
!pip install --upgrade soda-core-contracts
#!pip install soda-core-postgres
#!pip install soda-core-redshift
!pip install --upgrade dask
!pip install --upgrade soda-core-pandas-dask
!pip install python-dotenv
!pip install pyyaml

  Using cached dask-2024.7.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached importlib_metadata-8.2.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached zipp-3.19.2-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 0.6/1.2 MB 7.3 kB/s eta 0:01:21
ERROR: Exception:
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/poc_soda-1ZRiZgOA/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/home/ubuntu/.local/share/virtualenvs/poc_soda-1ZRiZgOA/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/home/ubuntu/.local/share/virtualenvs/poc_soda-1ZRiZgOA/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/home/ubuntu/.local/share/virtualenvs/poc_soda-1ZRiZgOA/lib/python3.10/site-p

In [5]:
import dask.dataframe as dd
from soda.scan import Scan

AttributeError: module 'pandas.core.strings' has no attribute 'StringMethods'

In [ ]:
# Create Soda Library Scan object and set a few required properties
scan = Scan()
scan.set_scan_definition_name("test")
scan.set_data_source_name("dask")

# Read a `cities` CSV file with columns 'city', 'population'
ddf = dd.read_csv('data/emoji_usage_dataset.csv')

scan.add_dask_dataframe(dataset_name="emoji", dask_df=ddf)

# Define checks using SodaCL

checks = """
checks for cities:
    - row_count > 0
"""

# Add the checks to the scan and set output to verbose
scan.add_sodacl_yaml_str(checks)

scan.set_verbose(True)

# Execute the scan
scan.execute()

# Inspect the scan object to review scan results
scan.get_scan_results()